# Matrix Generation

We need to generate matrices that are irreducible and aperiodic.

In [1]:
import math
import random
import pickle
import numpy as np
from tqdm import tqdm
from scipy.sparse import dok_matrix

In [2]:
def is_ergodic(matrix: dok_matrix, n: int) -> bool:
    """
    An ergodic matrix is aperiodic and irreducible. By Wielandt's theorem if when the matrix is multiplied by itself m
    times, where m = (n - 1) * (n - 1) + 1, and all its entries are positive then the matrix is ergodic. n is the number
    of sates.

    :param matrix: matrix to check
    :return: true if the matrix is ergodic, false otherwise
    """
    matrix = matrix.tocsr(copy=True)
    m = (n - 1) * (n - 1) + 1
    multiplicities = [matrix]
    for i in range(int(math.log(m, 2))):
        matrix = matrix.dot(matrix)
        multiplicities.append(matrix)
    index = len(multiplicities) - 1
    res = None
    while m > 0:
        if m & 1:
            if res is None:
                res = multiplicities[index]
            else:
                res = res.dot(multiplicities[index])
        index -= 1
        m = m >> 1
    return res.count_nonzero() == n * n

In [3]:
def is_irreducible(indexes):
    seen = set()
    i = 0
    while len(seen) != len(indexes):
        if indexes[i] == i:
            return False
        if indexes[i] in seen:
            return False
        seen.add(indexes[i])
        i = indexes[i]
    return True

In [4]:
# print(is_ergodic(dok_matrix([[0.7, 0.3, 0], [0, 0, 1], [0, 0.6, 0.4]]), 3))     # False
# print(is_ergodic(dok_matrix([[0.7, 0.3, 0], [0, 0.5, 0.5], [0.4, 0.6, 0]]), 3)) # True
# print(is_ergodic(dok_matrix([[0.7, 0.3, 0], [0, 0, 1], [0.4, 0.6, 0]]), 3))     # True
# print(is_ergodic(dok_matrix([[0, 1, 0], [0, 0, 1], [1, 0, 0]]), 3))             # False
# print(is_ergodic(dok_matrix([[0, 1, 0], [0.5, 0, 0.5], [0, 1, 0]]), 3))         # False
# print(is_ergodic(dok_matrix([[0, 0.5, 0.5], [0, 0.5, 0.5], [1, 0, 0]]), 3))     # True

In [5]:
def generate_1(n: int, sizes: list[int], p1: float, p2: float) -> list[list[list[int]]]:
    """
    Generate matrices with random values in them. The values are between 0 and 1. The amount of zeros on each row starts at p1 and ends p2.

    :param n: how many of each matrix size should be generated
    :param sizes: the sizes of the matrices that should be generated
    :return: the generated matrices
    """
    all = []
    increase = (p2 - p1) / n
    for size in tqdm(sizes):
        i = 0
        matrices = []
        zeros_p = p1
        while i < n:
            matrix = dok_matrix((size, size), dtype=np.float64)
            for j in range(size):
                s = 0
                while s == 0:
                    row = random.choices(range(1, 101), k=size)
                    zeros = random.sample(range(0, size), int(size * zeros_p))
                    for index in zeros:
                        row[index] = 0
                    s = sum(row)
                row = [(x / s) for x in row]
                matrix[j] = row
            if not is_ergodic(matrix, size):
                continue
            zeros_p += increase
            matrices.append(matrix)
            all.append(matrix)
            i += 1
        with open(f"./data/generated/matrices_{size}_{n}.pickle", 'wb') as f:
            pickle.dump(matrices, f)
    return all

In [6]:
def generate_2(n: int, sizes: list[int]) -> list[dok_matrix]:
    """
    Generate matrices with random values in them. The values are between 0 and 1. The amount of nonzero values on a row is between 1 and 11.

    :param n: how many of each matrix size should be generated
    :param sizes: the sizes of the matrices that should be generated
    :return: the generated matrices
    """
    all = []
    for size in tqdm(sizes):
        i = 0
        matrices = []
        transitions_on_row = np.random.randint(1, 11)
        while i < n:
            matrix = dok_matrix((size, size), dtype=np.float64)
            indices = np.random.choice(size, size=size, replace=False)
            while not is_irreducible(indices):
                indices = np.random.choice(size, size=size, replace=False)
            for (i_i, index) in enumerate(indices):
                matrix[i_i, index] = np.random.randint(1, 100)
            for _ in range(min(transitions_on_row, size)):
                for row in range(size):
                    col = np.random.randint(size)
                    if matrix[row, col] != 0:
                        continue
                    matrix[row, col] = np.random.randint(1, 100)
            for j in range(size):
                row = matrix[j].toarray()[0]
                s = sum(row)
                row = [x / s for x in row]
                matrix[j] = row
            if not is_ergodic(matrix, size):
                continue
            matrices.append(matrix)
            all.append(matrix)
            i += 1
        with open(f"./data/generated2/matrices_{size}_{n}.pickle", 'wb') as f:
            pickle.dump(matrices, f)
    return all

In [7]:
random.seed(4)
np.random.seed(4)
dataset1 = generate_1(10, range(10, 200), 0.25, 0.75)

random.seed(4)
np.random.seed(4)
dataset2 = generate_2(10, range(10, 200))

100%|██████████| 190/190 [04:38<00:00,  1.47s/it]
